<a href="https://colab.research.google.com/github/pddiste/pairs/blob/main/examples/stocks/stocks-trading-basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alpaca-py stock trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/stocks/stocks-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with stock trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for stocks as an example.

In [10]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PKHXGNJT1P9CBVISDPGT"
secret_key = "alfkHTbskBoUiI9wkGEtkrSKyjqlUT8cgHrKycfa"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [11]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [12]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [13]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [14]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [15]:
# check version of alpaca-py
alpaca.__version__

'0.42.0'

# Trading Client

In [18]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [19]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA397VSJ2LON',
    'accrued_fees': '0',
    'buying_power': '194627.4',
    'cash': '94851.99',
    'created_at': datetime.datetime(2022, 10, 23, 10, 2, 33, 917501, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100940.43',
    'id': UUID('3cb1b5ac-3657-4064-9408-3afc371e6d3f'),
    'initial_margin': '3626.73',
    'last_equity': '100911.40448533686',
    'last_maintenance_margin': '1817.82',
    'long_market_value': '6088.44',
    'maintenance_margin': '1826.53',
    'multiplier': '2',
    'non_marginable_buying_power': '95313.7',
    'options_approved_level': 3,
    'options_buying_power': '97313.7',
    'options_trading_level': 3,
    'pattern_day_trader': True,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100940.43',
    'regt_buying_power': '194627.4',
    '

In [20]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [21]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [22]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('b1cb0701-8610-4094-8de7-5dfe2bd4dc47'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Chanson International Holding Class A Ordinary Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'CHSN',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'fractionable': False,
     'id': UUID('9a20d343-2ed9-4ad0-8ef2-c7e6930001c1'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Chin

### Orders

In [23]:
# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [25]:
# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
req = MarketOrderRequest(
    symbol = symbol,
    qty = 5.5,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

APIError: {"code":40310000,"message":"fractional trading is disabled for this account"}

In [26]:
# simple, market order, notional
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is in the example above)
# or notional value (e.g. 100 USD) (which is in this example)
req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

APIError: {"code":40310000,"message":"fractional trading is disabled for this account"}

In [ ]:
# simple, limit order, fractional qty
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 550.25,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

In [ ]:
# stop order
req = StopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

In [ ]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 550,
                    stop_price = 600
                    )

res = trade_client.submit_order(req)
res

In [ ]:
# bracket order with both stop loss and take profit
req = MarketOrderRequest(
                    symbol = symbol,
                    qty = 5,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    order_class = OrderClass.BRACKET,
                    take_profit = TakeProfitRequest(limit_price=600),
                    stop_loss = StopLossRequest(stop_price=300)
)
res = trade_client.submit_order(req)
res

In [ ]:
# oto order with stop loss
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OTO,
                    stop_loss = StopLossRequest(stop_price = 300)
                    )

res = trade_client.submit_order(req)
res

In [ ]:
# oco limit order
req = LimitOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    limit_price = 500,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.DAY,
                    Class = OrderClass.OCO
                    )

res = trade_client.submit_order(req)
res

In [ ]:
# trailing stop order
req = TrailingStopOrderRequest(
                    symbol = symbol,
                    qty = 1,
                    side = OrderSide.SELL,
                    time_in_force = TimeInForce.GTC,
                    trail_percent = 0.20 # you can also use trail_price instead of trail_percent
                    )

res = trade_client.submit_order(req)
res

In [ ]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

In [ ]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

In [ ]:
# cancel all open orders
trade_client.cancel_orders()

### Positions

In [ ]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

In [ ]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [27]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [28]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high     low   close   volume  \
symbol timestamp                                                            
SPY    2025-07-21 08:00:00+00:00  629.11  629.56  629.05  629.39  65231.0   
       2025-07-21 09:00:00+00:00  629.38  629.59  629.30  629.31  41725.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2025-07-21 08:00:00+00:00       1101.0  629.287945  
       2025-07-21 09:00:00+00:00        630.0  629.446226

In [29]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price  size  \
symbol timestamp                                                 
SPY    2025-07-21 08:00:00.008280+00:00        K  629.00   2.0   
       2025-07-21 08:00:00.008286+00:00        K  628.64   3.0   

                                                     id conditions tape  
symbol timestamp                                                         
SPY    2025-07-21 08:00:00.008280+00:00  52983525027897  [ , T, I]    B  
       2025-07-21 08:00:00.008286+00:00  52983525027898  [ , T, I]    B

In [30]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SPY    2025-07-21 08:00:00.001144+00:00     629.08       1.0            T   
       2025-07-21 08:00:00.008386+00:00     629.08       1.0            T   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SPY    2025-07-21 08:00:00.001144+00:00     629.16       1.0            T   
       2025-07-21 08:00:00.008386+00:00     629.00       1.0            K   

                                        conditions tape  
symbol timestamp                                         
SPY    2025-07-21 08:00:00.001144+00:00        [R]    B  
       2025-07-21 08:00:00.008386+00:00        [R]    B

In [31]:
# get latest quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
)
res = stock_historical_data_client.get_stock_latest_quote(req)
res

{'SPY': {   'ask_exchange': 'V',
     'ask_price': 636.86,
     'ask_size': 12.0,
     'bid_exchange': 'V',
     'bid_price': 636.85,
     'bid_size': 1.0,
     'conditions': ['R'],
     'symbol': 'SPY',
     'tape': 'B',
     'timestamp': datetime.datetime(2025, 7, 25, 19, 59, 23, 208012, tzinfo=TzInfo(UTC))}}

# Market Data (Stream)

In [32]:
stock_data_stream_client = StockDataStream(api_key, secret_key, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

symbol='SPY' timestamp=datetime.datetime(2025, 7, 25, 19, 59, 29, 118264, tzinfo=datetime.timezone.utc) exchange='V' price=636.87 size=28.0 id=52983828988232 conditions=[' ', 'I'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 7, 25, 19, 59, 29, 118285, tzinfo=datetime.timezone.utc) bid_price=636.87 bid_size=6.0 bid_exchange='V' ask_price=636.88 ask_size=13.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 7, 25, 19, 59, 29, 118796, tzinfo=datetime.timezone.utc) bid_price=636.87 bid_size=2.0 bid_exchange='V' ask_price=636.88 ask_size=13.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 7, 25, 19, 59, 29, 119127, tzinfo=datetime.timezone.utc) bid_price=636.87 bid_size=2.0 bid_exchange='V' ask_price=636.88 ask_size=13.0 ask_exchange='V' conditions=['R'] tape='B'
symbol='SPY' timestamp=datetime.datetime(2025, 7, 25, 19, 59, 29, 119382, tzinfo=datetime.timezone.utc) bid_price=636.87 bid_size=6.0 bi

TimeoutError: 

# Corporate actions

In [33]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,id,symbol,cusip,rate,special,foreign,process_date,ex_date,record_date,payable_date
corporate_action_type,,,,,,,,,,
cash_dividends,e06f1a82-04b5-4b90-9c7b-06d22b720db1,SPY,78462F103,1.405559,False,False,2020-04-30,2020-03-20,2020-03-23,2020-04-30
cash_dividends,a6587d80-e08a-4b1a-aa94-857549880c59,SPY,78462F103,1.366242,False,False,2020-07-31,2020-06-19,2020-06-22,2020-07-31
cash_dividends,ffb8940e-53fe-4b3e-a475-e15e39561d3d,SPY,78462F103,1.339224,False,False,2020-10-30,2020-09-18,2020-09-21,2020-10-30
cash_dividends,4d35e709-773b-46d4-b292-34d14c46f287,SPY,78462F103,1.580000,False,False,2021-01-29,2020-12-18,2020-12-21,2021-01-29
cash_dividends,f1ddda1e-b10c-4576-8253-4d8d5a149de6,SPY,78462F103,1.277788,False,False,2021-04-30,2021-03-19,2021-03-22,2021-04-30
cash_dividends,a06ac608-4210-4e4f-94f0-4fa342a8e1fd,SPY,78462F103,1.375875,False,False,2021-07-30,2021-06-18,2021-06-21,2021-07-30
cash_dividends,d2b09118-e4c0-4b93-9f46-10b211400773,SPY,78462F103,1.428117,False,False,2021-10-29,2021-09-17,2021-09-20,2021-10-29
cash_dividends,7cdc9442-1a32-4f40-9d2a-b721edfef9e9,SPY,78462F103,1.636431,False,False,2022-01-31,2021-12-17,2021-12-20,2022-01-31
cash_dividends,f5ca31fd-da0d-4651-be6b-6ac78542259e,SPY,78462F103,1.366009,False,False,2022-04-29,2022-03-18,2022-03-21,2022-04-29
